# 以相似度为训练目标

## 生成训练数据，暂存于本地

In [0]:
!git clone https://github.com/daibiaoxuwu/05wan3

In [0]:
!git clone https://github.com/daibiaoxuwu/05wan3b

In [0]:
import os
import cv2
import numpy as np
path1 = '05wan3'
path2 = '05wan3b/0'

clocks = dict()
x_train0 = []
y_train0 = []
for filename in os.listdir(path2):
    if 'git' in filename: continue
    #print(path1)
    #ftimestr,number = path1.split('_')
    clocks[int(filename.split('_')[0])]=int(filename.split('_')[1][:-4])
    #clocks[int(ftimestr)]=int(number)
keys = clocks.keys()
for filename in os.listdir(path1):
    if 'git' in filename or 'c.jpg' in filename: continue
    ftime = int(filename.split('_')[0])
    value = max(filter(lambda t:t<ftime,keys))
    x_train0.append(cv2.imread(os.path.join(path1,filename), cv2.IMREAD_GRAYSCALE)/255.0)
    y_train0.append(clocks[value])
    
print(len(x_train0),len(y_train0))

In [0]:
import seaborn as sns
sns.countplot(y_train0)
print(y_train0[:100])

In [0]:

from matplotlib import pyplot as plt
lst = os.listdir(path1)
for i in range(5):
  plt.imshow(x_train0[i])
  plt.show()
  print(y_train0[i])
  print(lst[i])

# 转为onehot，分离test和train

# 装载Tensorflow

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

# 安装 TensorFlow
try:
  # Colab only
  %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf

# 输入数据生成器
# 变为5张图片一叠

In [0]:
idxes = []
for i in range(10):idxes.append([])
for idx, val in enumerate(y_train0):
  idxes[val].append(idx)
for i in range(10):
  print(idxes[i][0:2])


# 搭建模型和训练
现在的网络去掉dropout则可以拟合10张图片，
但是加上dropout就拟合不了。
从7张拟合5张的，则可以在两个epoch（每个epoch抽取60000次）内到80%。

继续训练20张，一个epoch就到60%。

In [0]:

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (5, 5),padding = 'Same',  activation='relu', input_shape=(12, 12, 5)),
    tf.keras.layers.Conv2D(32, (5, 5),padding = 'Same',  activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(64, (3, 3),padding = 'Same',  activation='relu'),
    tf.keras.layers.Conv2D(64, (3, 3),padding = 'Same',  activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    
    #tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    #tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10,activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


# 搭建训练数据

## 搭建Overfit数据：只有十叠按序的照片

In [0]:
import random
def train_generator():
  for train in range(60000):
    ans = random.randint(0,9)
    idx2 = idxes[ans][:5]
    data = np.stack([x_train0[ix] for ix in idx2],axis=2)
    yield data,ans

In [0]:
train_dataset = tf.data.Dataset.from_generator(train_generator,  (tf.float32, tf.float32), (tf.TensorShape([12,12,5]),tf.TensorShape([])))
train_dataset = train_dataset.batch(32)

In [0]:
model.fit(train_dataset,epochs=10)

# 两个epoch后达到99正确率。下面采用乱序的7张照片中取5张。

In [0]:
import random
def train_generator(slice):
  for train in range(60000):
    ans = random.randint(0,9)
    idx2 = random.sample(idxes[ans][:slice],5)
    data = np.stack([x_train0[ix] for ix in idx2],axis=2)
    ansy = np.zeros(10)
    ansy[ans] = 1
    yield data,ans

In [0]:
train_dataset = tf.data.Dataset.from_generator(train_generator,  (tf.float32, tf.float32), (tf.TensorShape([12,12,5]),tf.TensorShape([])), args=(7,))
train_dataset = train_dataset.batch(32)

In [0]:
model.fit(train_dataset,epochs=2)

In [0]:
tx,ty = next(train_generator())
for i in range(5):
  plt.imshow(tx[:,:,i])
  plt.show()
print(ty)
print(model.predict(np.array((tx,))))

# 训练并验证模型：